In [1]:
# load the required libraries
import scipy # to calculate correlation
import pandas as pd # to use dataframe
import seaborn as sns # to make plots
import matplotlib.pyplot as plt # to make plots
import numpy as np
from sklearn.metrics import mean_absolute_error # to calculate MAE
from sklearn.model_selection import train_test_split, KFold # to create data splits
from scipy import stats

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import shuffle
import joblib

from pathlib import Path
import os

import datetime

In [3]:
#### Define the conditions for which the analysis should run:

## Define analysis input of the structural volume tables:

## Define Time point
#time = 'Base'
time = '2yFU'
#time = '4yFU'

input_type = 'cortical_thickness_Desikan'

## Define whether the analysis is performed for the whole group, only boys or only girls:
group = 'whole_group'
#group = 'only_boys'
#group = 'only_girls'

confound = 'None'

## define analysis type:
analysis_type = 'scikit_learn'
#analysis_type = 'Julearn'

## define target variable:
#target = 'PhSeAbuse'
#target = 'NHT'
#target = 'PreSubExp'
target = 'HD'
#target = 'Scarcity'
#target = 'ParentPsych'
#target = 'SESrec'

#Available 2 year: Physical and Sexual Violence, Neighborhood Threat, Household Dysfunction, SES_rec

#number of repetitions you want to perform for the predictive models:
rep = 100


In [4]:
### define local data directory:

ABCD_data_directory = 'ABCD PATHNAME'

results_directory = 'RESULTSDIRECTORY PATHNAME'

### define new folder in results-directory:

#foldername = f"abcd_{time}_{target}_{group}_{input_type}"

foldername = f"abcd_UniqueSample_{target}_PredictAll_{time}_{group}_{input_type}"

full_path = os.path.join(results_directory,foldername)

Path(full_path).mkdir(parents=True, exist_ok=True)


In [5]:
#LOAD THE CORRECT FILE
FinalSample = pd.read_csv(results_directory +'FINAL SAMPLE PATHNAME', low_memory=False)


In [6]:
# split table for boys and girls:

# boys:
data_only_boys = FinalSample[FinalSample['demo_sex_v2'] == 1]

rows, cols = data_only_boys.shape
print(f"Dataframe containing only boys has {rows} rows and {cols} colums")

# girls:
data_only_girls = FinalSample[FinalSample['demo_sex_v2'] == 2]

rows, cols = data_only_girls.shape
print(f"Dataframe containing only girls has {rows} rows and {cols} colums")


Dataframe containing only boys has 2194 rows and 147 colums
Dataframe containing only girls has 1892 rows and 147 colums


In [7]:
### define which kind of input and tables to use:
# define group and split data into train and test:

if group == 'whole_group':
    data = FinalSample
    
elif group == 'only_boys':
    data = data_only_boys
    
elif group == 'only_girls':
    data = data_only_girls
    
sites = data['site_id_l']
sites

counts_values_sites = data['site_id_l'].value_counts()
counts_values_sites

site_id_l
site16    404
site10    290
site04    281
site20    277
site17    268
site13    232
site14    216
site06    216
site12    214
site21    180
site11    175
site07    168
site03    150
site18    144
site15    143
site02    134
site01    133
site19    129
site05    128
site09    122
site08     82
Name: count, dtype: int64

In [8]:
# define the type of input:

if input_type == 'volumes_Desikan':
    
    input_data = data.filter(regex='smri_vol_cdk').drop(columns=['smri_vol_cdk_totallh', 'smri_vol_cdk_totalrh', 'smri_vol_cdk_total'])


elif input_type == 'volumes_Destrieux':

    columns_to_select = [f'mrisdp_{i}' for i in range(454, 602)]
    input_data = data[columns_to_select]


elif input_type == 'volumes_subcortical_volumes_aseg':

    list_include_subcortical_regions = ['smri_vol_scs_aal', 'smri_vol_scs_aar',                 # accumbens
                                        'smri_vol_scs_amygdalalh', 'smri_vol_scs_amygdalarh',   # amygdala
                                        'smri_vol_scs_caudatelh', 'smri_vol_scs_caudaterh',     # caudate
                                        'smri_vol_scs_crbcortexlh', 'smri_vol_scs_crbcortexrh', # cerebellum cortex
                                        'smri_vol_scs_vedclh', 'smri_vol_scs_vedcrh',           # ventral diencephalon
                                        'smri_vol_scs_hpuslh', 'smri_vol_scs_hpusrh',           # hippocampus
                                        'smri_vol_scs_pallidumlh', 'smri_vol_scs_pallidumrh',   # pallidum
                                        'smri_vol_scs_putamenlh', 'smri_vol_scs_putamenrh',     # putamen
                                        'smri_vol_scs_tplh',  'smri_vol_scs_tprh',]             # thalamus

    input_data = data[list_include_subcortical_regions]


elif input_type == 'cortical_thickness_Desikan':

    input_data = data.filter(regex='smri_thick_cdk').drop(columns=['smri_thick_cdk_meanlh', 'smri_thick_cdk_meanrh', 'smri_thick_cdk_mean'])
    

elif input_type == 'cortical_thickness_Destrieux':

    columns_to_select = [f'mrisdp_{i}' for i in range(1, 149)]
    input_data = data[columns_to_select]


elif input_type == 'surface_area_Desikan':

    input_data = data.filter(regex='smri_area_cdk').drop(columns=['smri_area_cdk_totallh', 'smri_area_cdk_totalrh', 'smri_area_cdk_total'])
    

elif input_type == 'surface_area_Destrieux':

    columns_to_select = [f'mrisdp_{i}' for i in range(303, 451)]
    input_data = data[columns_to_select]


input_data

,smri_thick_cdk_banksstslh,smri_thick_cdk_cdacatelh,smri_thick_cdk_cdmdfrlh,smri_thick_cdk_cuneuslh,smri_thick_cdk_ehinallh,smri_thick_cdk_fusiformlh,smri_thick_cdk_ifpllh,smri_thick_cdk_iftmlh,smri_thick_cdk_ihcatelh,smri_thick_cdk_locclh,...,smri_thick_cdk_rracaterh,smri_thick_cdk_rrmdfrrh,smri_thick_cdk_sufrrh,smri_thick_cdk_suplrh,smri_thick_cdk_sutmrh,smri_thick_cdk_smrh,smri_thick_cdk_frpolerh,smri_thick_cdk_tmpolerh,smri_thick_cdk_trvtmrh,smri_thick_cdk_insularh
0,2.879,2.514,2.826,2.053,3.224,3.053,2.744,3.121,2.495,2.240,...,2.799,2.587,3.030,2.565,3.201,2.800,3.108,3.743,2.931,3.321
1,2.732,2.850,2.894,2.329,3.141,2.916,2.817,2.819,2.595,2.417,...,3.133,2.665,3.152,2.680,3.158,2.866,3.349,3.910,2.785,3.129
2,2.004,2.582,2.807,1.922,3.282,2.714,2.395,2.956,2.454,2.197,...,3.137,2.449,2.891,2.373,2.755,2.613,3.225,3.445,2.764,3.254
3,2.712,2.672,2.635,1.859,3.276,2.759,2.633,2.893,2.367,2.199,...,2.689,2.501,2.913,2.305,3.034,2.617,2.737,3.550,2.433,3.236
4,2.723,2.998,2.881,1.986,3.463,2.885,2.726,3.123,2.756,2.334,...,2.891,2.705,3.099,2.366,3.256,2.754,2.817,4.111,2.954,3.158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,2.856,2.711,2.844,2.037,3.065,2.709,2.750,2.739,2.558,2.289,...,2.717,2.634,2.925,2.458,3.067,2.860,2.792,4.194,2.731,3.112
4082,2.596,2.439,2.866,2.090,2.845,2.864,2.686,2.793,2.107,2.121,...,2.901,2.476,2.984,2.484,2.954,2.632,2.885,3.474,2.798,3.292
4083,2.796,2.652,2.993,1.867,3.153,2.756,2.802,2.968,2.182,2.299,...,2.602,2.621,3.065,2.544,2.995,2.758,2.974,3.823,2.884,3.242
4084,2.631,2.721,2.929,2.205,3.217,2.801,2.469,2.778,2.398,2.249,...,3.006,2.461,2.898,2.420,2.772,2.728,2.824,3.247,2.312,3.035


In [9]:
### correct for confounding variables if necessary:

if confound == 'ICV':

    # define list of variablenames to iterate through:
    variable_names = input_data.columns.tolist() 

    # Calculate the scaled values and save them in a new DataFrame called 'ICV_scaled_columns'
    ICV_scaled_columns = pd.DataFrame({
        f'{col}_ICV_scaled': input_data[col] / data['smri_vol_scs_intracranialv']
        for col in variable_names
    })

    X = ICV_scaled_columns


elif confound == 'None':
    X = input_data

X

,smri_thick_cdk_banksstslh,smri_thick_cdk_cdacatelh,smri_thick_cdk_cdmdfrlh,smri_thick_cdk_cuneuslh,smri_thick_cdk_ehinallh,smri_thick_cdk_fusiformlh,smri_thick_cdk_ifpllh,smri_thick_cdk_iftmlh,smri_thick_cdk_ihcatelh,smri_thick_cdk_locclh,...,smri_thick_cdk_rracaterh,smri_thick_cdk_rrmdfrrh,smri_thick_cdk_sufrrh,smri_thick_cdk_suplrh,smri_thick_cdk_sutmrh,smri_thick_cdk_smrh,smri_thick_cdk_frpolerh,smri_thick_cdk_tmpolerh,smri_thick_cdk_trvtmrh,smri_thick_cdk_insularh
0,2.879,2.514,2.826,2.053,3.224,3.053,2.744,3.121,2.495,2.240,...,2.799,2.587,3.030,2.565,3.201,2.800,3.108,3.743,2.931,3.321
1,2.732,2.850,2.894,2.329,3.141,2.916,2.817,2.819,2.595,2.417,...,3.133,2.665,3.152,2.680,3.158,2.866,3.349,3.910,2.785,3.129
2,2.004,2.582,2.807,1.922,3.282,2.714,2.395,2.956,2.454,2.197,...,3.137,2.449,2.891,2.373,2.755,2.613,3.225,3.445,2.764,3.254
3,2.712,2.672,2.635,1.859,3.276,2.759,2.633,2.893,2.367,2.199,...,2.689,2.501,2.913,2.305,3.034,2.617,2.737,3.550,2.433,3.236
4,2.723,2.998,2.881,1.986,3.463,2.885,2.726,3.123,2.756,2.334,...,2.891,2.705,3.099,2.366,3.256,2.754,2.817,4.111,2.954,3.158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,2.856,2.711,2.844,2.037,3.065,2.709,2.750,2.739,2.558,2.289,...,2.717,2.634,2.925,2.458,3.067,2.860,2.792,4.194,2.731,3.112
4082,2.596,2.439,2.866,2.090,2.845,2.864,2.686,2.793,2.107,2.121,...,2.901,2.476,2.984,2.484,2.954,2.632,2.885,3.474,2.798,3.292
4083,2.796,2.652,2.993,1.867,3.153,2.756,2.802,2.968,2.182,2.299,...,2.602,2.621,3.065,2.544,2.995,2.758,2.974,3.823,2.884,3.242
4084,2.631,2.721,2.929,2.205,3.217,2.801,2.469,2.778,2.398,2.249,...,3.006,2.461,2.898,2.420,2.772,2.728,2.824,3.247,2.312,3.035


In [10]:
# define the target variable:

if target == 'PhSeAbuse':
    y = data['PhSeAbuseSUM']
    
elif target == 'NHT':
    y = data['NeighborhoodThreat']

elif target == 'PreSubExp':
    y = data['PreSubExp']
    
elif target == 'HD':
    y = data['HouseholdDysfunction']

elif target == 'Scarcity':
    y = data['Scarcity']

elif target == 'ParentPsych':
    y = data['ParentalPsychopathologyrecoded']

elif target == 'SESrec':
    y = data['flipped_income']

y

0       0.0
1       1.0
2       1.0
3       3.0
4       0.0
       ... 
4081    0.0
4082    0.0
4083    0.0
4084    2.0
4085    1.0
Name: HouseholdDysfunction, Length: 4086, dtype: float64

In [11]:

#For Baseline recodedses
#y_applied = data[['PhSeAbuseSUM', 'NeighborhoodThreat', 'PreSubExp', 'HouseholdDysfunction', 'Scarcity', 'ParentalPsychopathologyrecoded','flipped_income']]
#y_applied_datanames= ['PhSeAbuse','NHT', 'PreSubExp', 'HD', 'Scarcity', 'ParentPsych', 'SESrec']

#including INR
#y_applied = data[['PhSeAbuseSUM', 'NeighborhoodThreat', 'PreSubExp', 'HouseholdDysfunction', 'Scarcity', 'ParentalPsychopathologyrecoded','flipped_income', 'income_to_thresh']]
#y_applied_datanames= ['PhSeAbuse','NHT', 'PreSubExp', 'HD', 'Scarcity', 'ParentPsych', 'SESrec', 'INR']

#For 2y FU (no parental psychopathology, no scarcity, no prenatal substance exposure (only from baseline)
y_applied = data[['PhSeAbuseSUM', 'NeighborhoodThreat', 'HouseholdDysfunction','flipped_income']]
y_applied_datanames= ['PhSeAbuse','NHT', 'HD', 'SESrec']

#For 4y FU (no parental psychopathology, no scarcity, no prenatal substance exposure (only from baseline)
#y_applied = data[['NeighborhoodThreat', 'HouseholdDysfunction','flipped_income']]
#y_applied_datanames= ['NHT', 'HD', 'SESrec']

In [12]:
y

0       0.0
1       1.0
2       1.0
3       3.0
4       0.0
       ... 
4081    0.0
4082    0.0
4083    0.0
4084    2.0
4085    1.0
Name: HouseholdDysfunction, Length: 4086, dtype: float64

In [13]:
y_applied

,PhSeAbuseSUM,NeighborhoodThreat,HouseholdDysfunction,flipped_income
0,0.0,8,0.0,4
1,0.0,4,1.0,10
2,0.0,4,1.0,4
3,0.0,1,3.0,1
4,0.0,5,0.0,1
...,...,...,...,...
4081,0.0,0,0.0,4
4082,0.0,4,0.0,3
4083,0.0,2,0.0,2
4084,0.0,3,2.0,8


In [14]:
full_path

'/Users/kbrosch/Desktop/ML/abcd_UniqueSample_HD_PredictAll_2yFU_whole_group_cortical_thickness_Desikan'

In [15]:
## save input data for the model to work with the data in the computation of the Null-distribution:
#Elvisha does it make sense to save y applied, too?
X.to_csv(os.path.join(full_path, "input_X.csv"), index=False)
y.to_csv(os.path.join(full_path, "input_y.csv"), index=False)
y_applied.to_csv(os.path.join(full_path, "input_y_applied.csv"), index=False)
sites.to_csv(os.path.join(full_path, "input_sites.csv"), index=False)

np.save((full_path + '/input_X.npy'),X)
np.save((full_path + '/input_Y.npy'),y)
np.save((full_path + '/input_y_applied.npy'),y_applied)
np.save((full_path + '/input_input_sites.npy'),sites)


In [16]:
### set up predictive models:
                               
# Create a Pipeline including Scaling and Ridge Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # normalize features; Standardize features by removing the mean and scaling to unit variance -> z = (x - u) / s
    ('ridge', Ridge(
            fit_intercept=True, # Whether to fit the intercept for this model. If set to false, no intercept will be used in calculations (i.e. X and y are expected to be centered).
            copy_X=True, # copy_Xbool, default=True; If True, X will be copied; else, it may be overwritten.
            max_iter=1000000, # Maximum number of iterations for conjugate gradient solver.
            #tol -> for ‘lsqr’: tol is set as atol and btol of scipy.sparse.linalg.lsqr, which control the norm of the residual vector in terms of the norms of matrix and coefficients. 
            solver='lsqr', # ‘lsqr’ uses the dedicated regularized least-squares routine scipy.sparse.linalg.lsqr. It is the fastest and uses an iterative procedure.
            positive=False, # default is false; When set to True, forces the coefficients to be positive. Only ‘lbfgs’ solver is supported in this case.
            random_state=None # default is None; Used when solver == ‘sag’ or ‘saga’ to shuffle the data.
    ))
])
    
    # Define the range of parameters for the GridSearch:

    #set hyperparameter grid space you want to search through for the model
    #adapted from the Thomas Yeo Lab Github: 
    #ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
param_grid = {
        'ridge__alpha': [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
              3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 2000, 
              3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    }

In [17]:
#create variables to store relevant data/output

#number of outcomes to predict
n_y_applied = y_applied.shape[1]


#create variables to store the results
#r^2 - coefficient of determination
r2_scores_list = np.zeros([n_y_applied, rep])
#explained variance
explained_variance_list = np.zeros([n_y_applied, rep])
#correlation between true and predicted (aka prediction accuracy)
corr_coeff_list_pearson = np.zeros([n_y_applied, rep])
corr_p_value_list_pearson = np.zeros([n_y_applied, rep])
#optimised alpha (hyperparameter)
#opt_alpha = np.zeros([n_y_applied, rep])
opt_alpha = []

#feature importance extracted from the model
#feature_names = X.columns
#feat_imp = np.zeros([n_y_applied, rep])
#feat_imp = []
#for when the feat weights get haufe-inverted
#featimp_haufe = np.zeros([n_y_applied, rep])
#featimp_haufe = []

#feature importance extracted from the model
feature_names = X.columns
feat_imp = pd.DataFrame(columns=feature_names)

# dataframe for feature importance with the specified number of columns
column_names = [f'Repetition{i+1}' for i in range(rep)]
featimp_haufev1_dataframe = pd.DataFrame(columns=column_names)


In [18]:
for p in range(rep):
    
        #print model # you're on
    print('Model %d' %(p+1))
    
        # Now you can use datetime functionalities
    current_time = datetime.datetime.now()
    print("Current Date and Time :",current_time)
    
        ### split data into training and test:
    
        # generate a single division (by using the next function) of the data X into training and test sets, with all data belonging to the same group site 
        # ending up either completely in the training set or in the test set. The indices of these data points are then stored 
        # in the variables train_inds and test_inds.
    train_indx, test_indx = next(GroupShuffleSplit(test_size=0.2, n_splits=1, random_state = p).split(X, groups=sites))
    
       # use .iloc[] to enable access to the rows of the DataFrame via the indices that were created with GroupShuffleSplit.
    X_train = X.iloc[train_indx]
    X_test = X.iloc[test_indx]
        
    y_train = y.iloc[train_indx] #applies to the previously selected target variable, i.e. y train
        
    sites_train = sites.iloc[train_indx]
    sites_test = sites.iloc[test_indx] 
    
        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
    best_scores = []
    best_params = []
    
       # Use GroupKFold for grouping according to sites
    group_kfold = GroupKFold(n_splits=5)

    print ("Optimising Models")
        
        # Performing the grid search with cross-validation taking into account the groups (sites)
    grid_search = GridSearchCV(pipeline, param_grid, cv=group_kfold, scoring='explained_variance')
    grid_search.fit(X_train, y_train, groups=sites_train)
    
        # Add the best Paramter and best score to the lists:
    best_params.append(grid_search.best_params_)
    best_scores.append(grid_search.best_score_)
    best_model = grid_search.best_estimator_
    
        # Print best Parameter and best score
    print("Best Parameter for this model: ", grid_search.best_params_)
    print("Best Score for this model: ", grid_search.best_score_)  
    print ("Evaluating Models")
    
        #save optimised alpha values
    opt_alpha.append(grid_search.best_params_['ridge__alpha'])

        ### predict on the test-data with the best performing model:
    
        # fit model:
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    
    
        # make preditions for the test set:
    y_pred = best_model.predict(X_test)
    
    # iterate through y_applied to exchange y_test:
    for nr_iteration in range(n_y_applied):
        y_applied_target = y_applied.iloc[:, nr_iteration]
        print ("y_test: %s" % y_applied.columns[nr_iteration])

        y_test = y_applied_target.iloc[test_indx] #iterates through all other variables, picks different y-applied, i.e., y test
            
        #reassign to loop
        #evaluate model:
        r2_score = best_model.score(X_test,y_test)
        r2_scores_list[nr_iteration, p] = r2_score
        print("r2-score: ",r2_score)

        #compute explained variance:
        ev_score = explained_variance_score(y_test, y_pred)
        explained_variance_list[nr_iteration, p] = ev_score
        print("explained variance score: ",ev_score)


    # compute correlation between true and predicted values (prediction accuracy):
        correlation_pearson, p_value_pearson = pearsonr(y_test, y_pred)

        corr_coeff_list_pearson[nr_iteration, p] = correlation_pearson
        corr_p_value_list_pearson[nr_iteration, p] = p_value_pearson
    
        print(f"Pearson-Correlation of y_test and y_pred: {correlation_pearson}")
        print(f"P-value: {p_value_pearson}")
    
    
    ### Extract feature weights of best model:
    
    ridge_regressor = best_model.named_steps['ridge']
    coefficients = ridge_regressor.coef_

    #feat_imp.append(coefficients)
    feat_imp.loc[p] = coefficients


    ### extract feature importance:
    #compute Haufe-inverted feature weights
    #preds = []
    y_train_preds = best_model.predict(X_train).ravel()
    
    #print ("Haufe-Transforming Feature Weights")
    cov_x = []
    cov_y = []
    
    #cov x is the covariance of the x_train data - covariance of brain data for the training set
    cov_x = np.cov(np.transpose(X_train))
    #cov y is the covariance of the y_train predicted data - covariance of the predicted behavior for the training set
    cov_y = np.cov(y_train_preds)
    #### What is the y_train_preds? Wo wird das gespeichert?? ist es das gleiche wie y_train? (Hier werden ja als Trainings-Datensatz eigentlich keine Predictions durchgefuhrt?)
    ### to be able to do the matrix multiplication using matmul, the matrix of the feature importance needed to be transposed:
    feat_imp_T = np.transpose(coefficients)
   
    #haufe-transformed feature weight is the cov x multiplied by the feature weights
    #(from the trained model based on th training data) and then divided by cov y
    #v1 here is based on eqn 6 of paper
    featimp_haufev1 = np.matmul(cov_x,feat_imp_T)*(1/cov_y)
    featimp_haufev1_dataframe[f'Repetition{p+1}'] = featimp_haufev1

# Print the current group and target in bold
print(f"\033[1m\033[32m\033[4m {target} in {group}, {input_type} \033[0m")

Model 1
Current Date and Time : 2025-05-10 23:14:19.233712
Optimising Models
Best Parameter for this model:  {'ridge__alpha': 10000}
Best Score for this model:  -0.0006841593940248192
Evaluating Models
y_test: PhSeAbuseSUM
r2-score:  -6.964276910353372
explained variance score:  -0.008639381557939751
Pearson-Correlation of y_test and y_pred: 0.058335625953021567
P-value: 0.08622147903472291
y_test: NeighborhoodThreat
r2-score:  -1.0281884131242842
explained variance score:  -0.00019426739124095427
Pearson-Correlation of y_test and y_pred: 0.0008715928578341414
P-value: 0.9795667459411564
y_test: HouseholdDysfunction
r2-score:  0.0016896105338259915
explained variance score:  0.0021810193414243395
Pearson-Correlation of y_test and y_pred: 0.04803359553169603
P-value: 0.15786153704585226
y_test: flipped_income
r2-score:  -1.2898134396879684
explained variance score:  0.003563251029491399
Pearson-Correlation of y_test and y_pred: 0.1446972620922226
P-value: 1.9156887357122708e-05
Model 2


In [19]:
 #### save results:
df_opt_alpha = pd.DataFrame(opt_alpha)
df_opt_alpha.to_csv(os.path.join(full_path, "optimised_alpha.csv"), index=False)

df_r2_scores_list = pd.DataFrame(r2_scores_list)
df_r2_scores_list.to_csv(os.path.join(full_path, "r2_values.csv"), index=False)

df_explained_variance_list = pd.DataFrame(explained_variance_list)
df_explained_variance_list.to_csv(os.path.join(full_path, "explained_variance.csv"), index=False)

df_corr_coeff_list_pearson = pd.DataFrame(corr_coeff_list_pearson)
df_corr_coeff_list_pearson.to_csv(os.path.join(full_path, "corr_coeff_list_pearson.csv"), index=False)

df_corr_p_value_list_pearson = pd.DataFrame(corr_p_value_list_pearson)
df_corr_p_value_list_pearson.to_csv(os.path.join(full_path, "corr_p_value_list_pearson.csv"), index=False)

feat_imp.to_csv(os.path.join(full_path, "feature_weights.csv"), index=False)

featimp_haufev1_dataframe.to_csv(os.path.join(full_path, "feature_weights_haufe_transformed_v1_equation_6.csv"), index=False)
np.save((full_path + '/feature_weights_haufe_transformed_v1_equation_6.npy'),featimp_haufev1_dataframe)
    

In [20]:
### compute means for each variable in y_applied:

# Pre-allocate DataFrames/Series with the correct index
n_iterations = len(y_applied_datanames)  # Or however many iterations you have
index = range(n_iterations) # Or a more meaningful index if available

mean_corr_coeff_pearson = pd.Series(index=index, dtype=float)
mean_explained_variance = pd.Series(index=index, dtype=float)
mean_r2_values = pd.Series(index=index, dtype=float)

for nr_iteration in range(n_iterations):

    mean_corr_coeff_pearson.iloc[nr_iteration] = df_corr_coeff_list_pearson.iloc[nr_iteration].mean()
    mean_explained_variance.iloc[nr_iteration] = df_explained_variance_list.iloc[nr_iteration].mean()
    mean_r2_values.iloc[nr_iteration] = df_r2_scores_list.iloc[nr_iteration].mean()


# Create a dictionary of the calculated means
data = {
    'Pearson Mean': mean_corr_coeff_pearson,
    'Explained Variance Mean': mean_explained_variance,
    'R-squared Mean': mean_r2_values
}

# Create the DataFrame
df_means = pd.DataFrame(data)

# Rename rows (index) baseline
#new_row_names = ['PhSeAbuse','NHT', 'PreSubExp', 'HD', 'Scarcity', 'ParentPsych', 'SESrec']
#df_means.index = new_row_names

# Rename rows (index) baseline INR
#new_row_names = ['PhSeAbuse','NHT', 'PreSubExp', 'HD', 'Scarcity', 'ParentPsych', 'SESrec', 'INR']
#df_means.index = new_row_names

#Rename rows (index) 2yFU
new_row_names = ['PhSeAbuse','NHT', 'HD', 'SESrec']
df_means.index = new_row_names

# Rename rows (index) 4yFUeb
#new_row_names = ['NHT', 'HD', 'SESrec']
#df_means.index = new_row_names

print(f"\033[1m\033[32m\033[4m {target} in {group}, {input_type} for {time} with {rep} reps.  \033[0m") 
print(df_means)
df_means.to_csv(os.path.join(full_path , "overview_means_corrcoef_explained_variance_r2.csv"), index=True)

 HD in whole_group, cortical_thickness_Desikan for 2yFU with 100 reps.  
           Pearson Mean  Explained Variance Mean  R-squared Mean
PhSeAbuse      0.026157                -0.010633       -2.942053
NHT            0.064615                 0.001793       -1.008257
HD             0.031151                 0.000051       -0.005048
SESrec         0.147865                 0.004193       -1.725404


In [21]:
print(f"\033[1m\033[32m\033[4m {target} in {group}, {input_type} for {time}. Now run step 2 \033[0m") 

 HD in whole_group, cortical_thickness_Desikan for 2yFU. Now run step 2 


In [22]:
total_sample_size = len(sites)
print("Sample size")
total_sample_size

Sample size


4086